In [1]:
%%capture
!pip install -q langchain==0.0.152 pyllamacpp==1.0.7 sentencepiece gpt4all

In [2]:
import requests
from pathlib import Path
from tqdm import tqdm

from langchain.llms import GPT4All
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [3]:
local_path = './models/gpt4all-lora-quantized-ggml.bin'
Path(local_path).parent.mkdir(parents=True, exist_ok=True)

url = 'https://the-eye.eu/public/AI/models/nomic-ai/gpt4all/gpt4all-lora-quantized-ggml.bin'

# send a GET request to the URL to download the file.
response = requests.get(url, stream=True)

# open the file in binary mode and write the contents of the response
# to it in chunks.
with open(local_path, 'wb') as f:
    for chunk in tqdm(response.iter_content(chunk_size=8192)):
        if chunk:
            f.write(chunk)

514266it [07:17, 1176.01it/s]


In [4]:
!git clone https://github.com/ggerganov/llama.cpp.git
!cd llama.cpp && git checkout 2b26469
!python3 llama.cpp/convert.py ./models/gpt4all-lora-quantized-ggml.bin

Cloning into 'llama.cpp'...
remote: Enumerating objects: 14571, done.
remote: Counting objects: 100% (4617/4617), done.
remote: Compressing objects: 100% (282/282), done.
remote: Total 14571 (delta 4462), reused 4386 (delta 4332), pack-reused 9954
Receiving objects: 100% (14571/14571), 16.47 MiB | 22.98 MiB/s, done.
Resolving deltas: 100% (10187/10187), done.
Note: switching to '2b26469'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 2b26469 convert.py: Support models which are stored in a single pytorch_mode

In [5]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template,
                        input_variables=['question'])

In [6]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = GPT4All(model="/content/models/ggml-model-q4_0.bin",
              callback_manager=callback_manager,
              verbose=True)
llm_chain = LLMChain(prompt = prompt, llm=llm)

In [7]:
question = "What happens when it rains somewhere?"
llm_chain.run(question)

 Question: What happens when it rains somewhere?

Answer: Let's think step by step. When there is rain, firstly the earth gets water from clouds and finally reaching land through precipitation which also includes snowfall or sleet as well if that area has mountains like Himalayas. In places where trees are found, they get their nutrition through airborne pollens too (if it's spring season) in addition to other factors such as evaporation and transpiration of plants themselves can also play a key role for rainfall formation!

" Question: What happens when it rains somewhere?\n\nAnswer: Let's think step by step. When there is rain, firstly the earth gets water from clouds and finally reaching land through precipitation which also includes snowfall or sleet as well if that area has mountains like Himalayas. In places where trees are found, they get their nutrition through airborne pollens too (if it's spring season) in addition to other factors such as evaporation and transpiration of plants themselves can also play a key role for rainfall formation!"

In [8]:
##writing poems
question = "Write a poem about friendship that rhymes."
llm_chain.run(question)

 Question: Write a poem about friendship that rhymes.

Answer: Let's think step by step.  First, we need to come up with some ideas for the theme of our Friendship Rhyme Poem which is "friendship." Well! I could start out my friendships journey by mentioning how happy it makes me when they are around sharing their warmth and cheerfulness that radiates like a starlight in darkest night. Then, we can move on to describe the ways our friends bring joy into our lives--whether through laughter or music, books or movies. As I write this poem about friendship rhymes let's also include some examples of how they support and motivate us when life gets tough! And finally, don’t forget a thoughtful ending that reminds the reader to treasure their friends while they still have them because goodbye is never an easy decision.

' Question: Write a poem about friendship that rhymes.\n\nAnswer: Let\'s think step by step.  First, we need to come up with some ideas for the theme of our Friendship Rhyme Poem which is "friendship." Well! I could start out my friendships journey by mentioning how happy it makes me when they are around sharing their warmth and cheerfulness that radiates like a starlight in darkest night. Then, we can move on to describe the ways our friends bring joy into our lives--whether through laughter or music, books or movies. As I write this poem about friendship rhymes let\'s also include some examples of how they support and motivate us when life gets tough! And finally, don’t forget a thoughtful ending that reminds the reader to treasure their friends while they still have them because goodbye is never an easy decision.'